In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
#export
from collections import OrderedDict
import glob
import os
from operator import itemgetter
import json
from multiprocessing import cpu_count
import re
import sys
from pdb import set_trace
from pprint import pprint as pp
import warnings
warnings.filterwarnings('ignore')

import ancli
from imageio import imread
from jupytools import auto_set_trace, is_notebook
import numpy as np
import pandas as pd
import PIL.Image

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as T
from torchvision.transforms.functional import to_tensor
from tqdm import tqdm
import pretrainedmodels
from visdom import Visdom

from catalyst.contrib.schedulers import OneCycleLR
from catalyst.data.dataset import ListDataset
from catalyst.dl.callbacks import AccuracyCallback, AUCCallback, F1ScoreCallback
from catalyst.dl.runner import SupervisedRunner
from catalyst.utils import get_one_hot

try:
    extended
except NameError:
    sys.path.insert(0, 'rxrx1-utils')
    import rxrx.io as rio
    
from basedir import ROOT, TRAIN, TEST, SAMPLE, NUM_CLASSES

W0831 20:10:45.944396 140527749310272 compression.py:14] lz4 not available, disabling compression. To install lz4, run `pip install lz4`.


In [3]:
#export
seed = 1
dev_id = 0
device = torch.device(dev_id)
set_trace = auto_set_trace()
# os.environ['CUDA_VISIBLE_DEVICES'] = f'{dev_id}'

Version of set_trace(): ipdb


## Dataset Definition

### Load meta-information

In [4]:
#export
def list_files(folder):
    dirname = os.path.expanduser(folder)
    return [os.path.join(dirname, x) for x in os.listdir(dirname)]

In [5]:
#export
def load_data():
    content = []
    for filename in ('train', 'test'):
        with open(f'{filename}.json') as f:
            content.append(json.load(f))
    return content

### Dataset class

In [6]:
#export
class RxRxDataset(Dataset):
    
    def __init__(self, items, onehot=True,
                 label_smoothing=None,
                 open_fn=PIL.Image.open, tr=None):
        
        super().__init__()
        self.onehot = onehot
        self.label_smoothing = label_smoothing
        self.open_fn = open_fn
        self.tr = tr
        self.targets = [item['sirna'] for item in items]
        self.images = [item['images'] for item in items]
        self.num_classes = len(np.unique(self.targets))
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        bunch = self.images[index]
        channels = []
        for i, filename in sorted(bunch, key=itemgetter(0)):
            img = self.open_fn(filename)
            img = img if self.tr is None else self.tr(img)
            channels.append(img)
        sample = torch.cat(channels, dim=0)
        y = self.targets[index]
        sample = dict(features=sample, targets=y)
        if self.onehot:
            y_enc = get_one_hot(
                y, smoothing=self.label_smoothing, 
                num_classes=self.num_classes)
            sample['targets_one_hot'] = y_enc
        return sample

### Data loaders

In [7]:
#export
def create_data_loaders(train_records, test_records, batch_size=4):
    ys = [r['sirna'] for r in train_records]
    train, valid = train_test_split(train_records, stratify=ys, test_size=0.1)
    trn_ds = RxRxDataset(train, tr=T.Compose([
        T.Resize(224),
        T.ToTensor(),
        T.Normalize((0.5,), (0.5,))
    ]))
    val_ds = RxRxDataset(valid, tr=T.Compose([
        T.Resize(224),
        T.ToTensor(),
        T.Normalize((0.5,), (0.5,))
    ]))
    tst_ds = RxRxDataset(test_records, onehot=False, tr=T.Compose([
        T.Resize(224),
        T.ToTensor(),
        T.Normalize((0.5,), (0.5,))
    ]))
    trn_dl = DataLoader(
        trn_ds, shuffle=True, 
        batch_size=batch_size, num_workers=cpu_count())
    val_dl = DataLoader(
        val_ds, shuffle=False,
        batch_size=batch_size, num_workers=cpu_count())
    tst_dl = DataLoader(
        tst_ds, shuffle=False,
        batch_size=batch_size, num_workers=cpu_count())
    loaders = OrderedDict()
    loaders['train'] = trn_dl
    loaders['valid'] = val_dl
    loaders['test'] = tst_dl
    return {'loaders': loaders, 'num_classes': NUM_CLASSES}

## Model Definition

### Meta

In [8]:
#export
model_name = 'resnet34'

### Functions 

In [9]:
#export
def plus_noise(t, std=0.001):
    return t + torch.randn(t.shape)*std

In [10]:
#export
def get_model(model_name, num_classes, pretrained='imagenet'):
    model_fn = pretrainedmodels.__dict__[model_name]
    model = model_fn(num_classes=1000, pretrained=pretrained)
    dim_feats = model.last_linear.in_features
    model.last_linear = nn.Linear(dim_feats, num_classes)
    new_conv = nn.Conv2d(6, 64, 7, 2, 3, bias=False)
    new_conv.weight.data[:,0:3,:] = plus_noise(model.conv1.weight.data.clone())
    new_conv.weight.data[:,3:6,:] = plus_noise(model.conv1.weight.data.clone())
    model.conv1 = new_conv
    return model

## Train

### Helper utils

In [11]:
#export
class RollingLoss:
    def __init__(self, smooth=0.98):
        self.smooth = smooth
        self.prev = 0
    def __call__(self, curr, batch_no):
        a = self.smooth
        avg_loss = a*self.prev + (1 - a)*curr
        debias_loss = avg_loss/(1 - a**batch_no)
        self.prev = avg_loss
        return debias_loss

In [12]:
#export
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False

In [13]:
#export
def get_layer(model, key):
    """Gets model layer using a key.
    
    The key could be hierarchical, like first.second.third where
    each dot separates hierarchy level.
    """
    parts = key.split('.')
    block = model
    for part in parts:
        block = getattr(block, part)
    return block

In [14]:
#export
def unfreeze_layers(model, names):
    for name in names:
        layer = get_layer(model, name)
        print(f'Unfreezing layer {name}')
        for param in layer.parameters():
            param.requires_grad = True

### Loop

In [15]:
#export
def train(epochs: int=1,
          batch_size: int=800, 
          model_name: str='resnet34', 
          logdir: str='/tmp/loops/',
          lrs: tuple=(1e-5, 1e-3, 5e-3),
          eta_min: float=1e-6,
          dev_id: int=1,
          visdom_host: str='0.0.0.0',
          visdom_port: int=9001):
    
    vis = Visdom(server=visdom_host, port=visdom_port,
                 username=os.environ['VISDOM_USERNAME'],
                 password=os.environ['VISDOM_PASSWORD'])
    
    experiment_id = f'{model_name}_e{epochs}_b{batch_size}'
    device = torch.device(f'cuda:{dev_id}')
    dataset = create_data_loaders(*load_data(), batch_size=batch_size)
    model = get_model(model_name, dataset['num_classes']).to(device)
    freeze_model(model)
    unfreeze_layers(model, ['conv1', 'bn1', 'layer4', 'last_linear'])
    
    loss_fn = nn.CrossEntropyLoss()
    conv, layer, head = lrs
    opt = torch.optim.SGD([
        {'params': model.conv1.parameters(), 'lr': conv},
        {'params': model.layer4.parameters(), 'lr': layer},
        {'params': model.last_linear.parameters(), 'lr': head}
    ])
    logdir = os.path.join(logdir, experiment_id)
    sched = CosineAnnealingWarmRestarts(
        opt, T_0=len(dataset['loaders']['train']), T_mult=2, eta_min=eta_min)
    rolling_loss = RollingLoss()
    os.makedirs(logdir, exist_ok=True)
    iteration = 0
    
    for epoch in range(1, epochs+1):
        trn_dl = dataset['loaders']['train']
        n = len(trn_dl)

        model.train()
        with tqdm(total=n) as bar:
            for i, batch in enumerate(trn_dl, 1):
                iteration += 1
                if i % 25 == 0:
                    for j, g in enumerate(opt.param_groups):
                        vis.line(X=[iteration], Y=[g['lr']], 
                                 win=f'metrics{j}', name=f'lr{j}', update='append')
                bar.set_description(f'[epoch:{epoch}/{epochs}][{i}/{n}]')
                opt.zero_grad()
                x = batch['features'].to(device)
                y = batch['targets'].to(device)
                out = model(x)
                loss = loss_fn(out, y)
                loss.backward()
                avg_loss = rolling_loss(loss.item(), i+1)
                opt.step()
                sched.step()
                bar.set_postfix(avg_loss=f'{avg_loss:.3f}')
                bar.update(1)
                vis.line(X=[iteration], Y=[avg_loss],
                         win='loss', name='avg_loss', update='append')

        val_dl = dataset['loaders']['valid']
        n = len(val_dl)

        model.eval()
        with torch.no_grad():
            matches = []
            with tqdm(total=n) as bar:
                for batch in val_dl:
                    x = batch['features'].to(device)
                    y = batch['targets'].to(device)
                    out = model(x)
                    y_pred = out.softmax(dim=1).argmax(dim=1)
                    matched = (y == y_pred).detach().cpu().numpy().tolist()
                    matches.extend(matched)
                    bar.update(1)
            acc = np.mean(matches)
            vis.line(X=[epoch], Y=[acc], win='acc', name='val_acc', update='append')
            print(f'validation accuracy: {acc:2.2%}')
            acc_str = str(int(round(acc * 10_000, 0)))
            path = os.path.join(logdir, f'train.{epoch}.{acc_str}.pth')
            torch.save(model.state_dict(), path)

In [16]:
#export
if __name__ == '__main__':
    if not is_notebook():
        ancli.make_cli(train)

In [17]:
train(model_name='resnet34', batch_size=256)

W0831 20:10:46.773514 140527749310272 __init__.py:505] Setting up a new session...
  0%|          | 0/286 [00:00<?, ?it/s]

Unfreezing layer conv1
Unfreezing layer bn1
Unfreezing layer layer4
Unfreezing layer last_linear


100%|██████████| 32/32 [00:27<00:00,  2.67it/s]


validation accuracy: 0.14%


NameError: name 'resnet' is not defined

## Old

In [ ]:
#export
epochs = 30
batch_size = 800
dataset = create_data_loaders(*load_data(), batch_size=batch_size)
model = get_model(model_name, dataset['num_classes']).to(device)
freeze_model(resnet)

for param in resnet.parameters():
    param.requires_grad = False
resnet.conv1.requires_grad = False
resnet.last_linear.weight.requires_grad = True
for param in resnet.layer4.parameters():
    param.requires_grad = True

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD([
    {'params': resnet.conv1.parameters(), 'lr': 1e-6},
    {'params': resnet.layer4.parameters(), 'lr': 1e-3},
    {'params': resnet.last_linear.parameters(), 'lr': 5e-3}
])

# logdir = '/tmp/protein/logs/'
logdir = '/tmp/loops/protein/'
runner = SupervisedRunner()
sched = CosineAnnealingWarmRestarts(opt, len(dataset['loaders']['train']), eta_min=1e-7)
rolling_loss = RollingLoss()
os.makedirs(logdir, exist_ok=True)

for epoch in range(epochs):
    trn_dl = dataset['loaders']['train']
    n = len(trn_dl)
    
    resnet.train()
    with tqdm(total=n) as bar:
        for i, batch in enumerate(trn_dl, 1):
            bar.set_description(f'[epoch:{epoch}][{i}/{n}]')
            opt.zero_grad()
            x = batch['features'].to(device)
            y = batch['targets'].to(device)
            out = resnet(x)
            loss = loss_fn(out, y)
            loss.backward()
            avg_loss = rolling_loss(loss.item(), i+1)
            opt.step()
            sched.step()
            bar.set_postfix(avg_loss=f'{avg_loss:.3f}')
            bar.update(1)
            
    val_dl = dataset['loaders']['valid']
    n = len(val_dl)
    
    resnet.eval()
    with torch.no_grad():
        matches = []
        with tqdm(total=n) as bar:
            for batch in val_dl:
                x = batch['features'].to(device)
                y = batch['targets'].to(device)
                out = resnet(x)
                y_pred = out.softmax(dim=1).argmax(dim=1)
                set_trace()
                matched = (y == y_pred).detach().cpu().numpy().tolist()
                matches.extend(matched)
                bar.update(1)
        acc = np.mean(matches)
        print(f'validation accuracy: {acc:2.2%}')
        acc_str = str(int(round(acc * 10_000, 0)))
        torch.save(resnet.state_dict(), os.path.join(logdir, f'train.{epoch}.{acc_str}.pth'))

In [ ]:
# TODO: report issue with one-hot smoothing AUC and accuracy metrics

## Test

In [ ]:
model = get_model(model_name, NUM_CLASSES)
checkpoints = []
checkpoints_path = '/tmp/loops/protein/'
for filename in os.listdir(checkpoints_path):
    _, acc, _, _ = filename.split('.')
    checkpoints.append((os.path.join(checkpoints_path, filename), int(acc)))
checkpoints.sort(key=itemgetter(1))
best, _ = checkpoints[-1]
model.load_state_dict(torch.load(best))
model.eval()
model.to(device)
for param in model.parameters():
    param.requires_grad = False

In [ ]:
dataset = create_data_loaders(*load_data(), batch_size=512)

In [ ]:
from tqdm import tqdm_notebook as tqdm
preds = []
for batch in tqdm(dataset['loaders']['test']):
    out = model(batch['features'].to(device))
    y = out.softmax(dim=1)
    preds.extend(y.tolist())

In [ ]:
filenames = sorted(list_files('~/data/protein/tmp/test'))

# odd
site1 = []
for filename, pred in list(zip(filenames, preds))[::2]:
    basename, _ = os.path.splitext(os.path.basename(filename))
    sirna = int(basename.split('_')[-1])
    if sirna != 0: 
        continue
    site1.append(pred)
    
# even
site2 = []
for filename, pred in list(zip(filenames, preds))[1::2]:
    basename, _ = os.path.splitext(os.path.basename(filename))
    sirna = int(basename.split('_')[-1])
    if sirna != 0: 
        continue
    site2.append(pred)

In [ ]:
t1 = torch.tensor(site1)
t2 = torch.tensor(site2)
avg_pred = ((t1 + t2)/2).argmax(dim=1)
print(avg_pred.shape)

In [ ]:
sample = pd.read_csv('/home/ck/data/protein/sample_submission.csv')
sample['sirna'] = avg_pred.tolist()
sample.to_csv('submit.csv', index=False)
from IPython.display import FileLink
FileLink('submit.csv')

In [ ]:
# #export
# print('Saving the trained model')
# basedir = os.path.expanduser('~/data/protein/tmp/models')
# os.makedirs(basedir)
# torch.save(resnet, os.path.join(basedir, 'resnet50_simple.pth'))